In [1]:
import tensorflow as tf
import numpy as np
import cv2
from wandb.keras import WandCallback
wandb.init(project="googlenet")

ImportError: cannot import name 'WandCallback' from 'wandb.keras' (C:\Users\chris\AppData\Local\Programs\Python\Python37\lib\site-packages\wandb\keras\__init__.py)

In [10]:
(X_train,y_train),(X_val,y_val)=tf.keras.datasets.cifar10.load_data()
print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
y_train=tf.keras.utils.to_categorical(y_train, num_classes=10)
y_val=tf.keras.utils.to_categorical(y_val, num_classes=10)
print(y_train.shape)



X_train: (50000, 32, 32, 3)
y_train: (50000, 1)
(50000, 10)


In [16]:
def generator_numpy(x,y,batch_size=64,shape=(151,151)):  #x.shape(32,32)->shape(x.shape[1],x.shape[0])
    #y=tf.keras.backend.batch_flatten(y)
    num=x.shape[0]
    while True:
        for i in range(0,num,batch_size):
            batch_x=np.array([cv2.resize(i,shape) for i in x[i:i+batch_size]],dtype='float32')
            batch_y=np.array(y[i:i+batch_size],dtype='int32')
            batch_x/=255
            yield batch_x,batch_y;

g_n=generator_numpy(X_train,y_train)
x,y=next(g_n)
print(x.shape)
print(y.shape)

(64, 151, 151, 3)
(64, 10)


In [ ]:
base_model=tf.keras.applications.InceptionV3(include_top=False,weights='imagenet',input_shape=(151,151,3))
base_model.summary()   
base_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'], callbacks=[WandbCallback()])
base_model.fit()

In [17]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=30,kernel_size=(3,3),input_shape=(40,40,3),activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'], callbacks=[WandbCallback()])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 38, 38, 30)        840       
_________________________________________________________________
flatten_2 (Flatten)          (None, 43320)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               4332100   
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 4,333,950
Trainable params: 4,333,950
Non-trainable params: 0
_________________________________________________________________


In [18]:
#model.fit_generator(generator_numpy(X_train,y_train,shape=(32,32)),steps_per_epoch=X_train.shape[0]/64,epochs=2,validation_data=generator_numpy(X_val,y_val,shape=(32,32)),validation_steps=X_val.shape[0]/64)
model.fit_generator(generator_numpy(X_train,y_train,shape=(40,40)),steps_per_epoch=X_train.shape[0]/64,epochs=2)


Epoch 1/2
745/781 [===========================>..] - ETA: 2s - loss: 1.6140 - accuracy: 0.4298

KeyboardInterrupt: 

In [9]:
datagen=tf.keras.preprocessing.image.ImageDataGenerator()
train=datagen.flow(X_train,y_train,shuffle=False,batch_size=64)
X,Y=next(train)
print(X.dtype)
print((X==x).all())
print((Y==y).all())

float32
True
True


TypeError: 'NumpyArrayIterator' object is not callable